<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_PIMS_01_Autentica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plant Information Management System (PIMS) - Autenticação

**Objetivo**

Criar sistema de login para um PIMS simples.

**Referências**

https://www.digitalocean.com/community/tutorials/how-to-add-authentication-to-your-app-with-flask-login-pt

## Modelo de Páginas

In [ ]:
!wget https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/ECAA07_PIMS_templates.zip
!unzip -q ECAA07_PIMS_templates.zip
!rm ECAA07_PIMS_templates.zip

## Bibliotecas

### Instalar

In [ ]:
!pip install flask flask-sqlalchemy flask-login flask_ngrok

### Utilizadas

In [3]:
# classe de aplicação web em flask
from flask import Flask
# utiliza modelos em páginas web
from flask import render_template
# redireciona o navegador para outra url
from flask import redirect
# gera a url de uma rota do flask
from flask import url_for
# coleta as variáveis passadas pelo navegador
from flask import request
# envia um valor de retorno para o conteúdo da página
from flask import flash
# gera um hash criptografado de uma senha
from werkzeug.security import generate_password_hash
# compara hashs de senha atual e gravada
from werkzeug.security import check_password_hash
# classe getora de banco de dados 
from flask_sqlalchemy import SQLAlchemy
# modelo de usuário para login
from flask_login import UserMixin
# classe gerenciadora de login para automação
from flask_login import LoginManager
# indica rotas que necessitam de login para entrar
from flask_login import login_required
# grava o usuário que logou na sessão
from flask_login import login_user
# obtem usuário logado na sessão
from flask_login import current_user
# apaga usuário logado da sessão
from flask_login import logout_user
# gerador de código aleatório para chave de segurança
from os import urandom
# permite ao colab gerar uma página externa para acessar a aplicação
from flask_ngrok import run_with_ngrok

## Criando aplicação Web

In [ ]:
# cria a aplicação web e pasta com modelos de páginas
app = Flask('PIMS UNIFEI', template_folder='templates')

# chave única do site para assinatura dos cookies
app.config['SECRET_KEY'] = urandom(12)
app

### Gestor de Banco de Dados

In [ ]:
# gerenciamento de banco de dados
db = SQLAlchemy()

# indica banco de dados da aplicação em sqlite3
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///pims.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # tira notificações do db

#conecta gerenciador de banco na aplicação
db.init_app(app)

# modelo para objeto usuário no banco
class Usuario(UserMixin, db.Model):
  id = db.Column(db.Integer, primary_key=True)
  usuario = db.Column(db.String(255))
  email = db.Column(db.String(150), unique=True)
  senha = db.Column(db.String(50))

# monta arquivo de banco, caso não exista
with app.app_context():
  db.create_all()
  
db

### Autenticação

In [ ]:
# gestão de login automatizada
login_manager = LoginManager()
login_manager.login_view = 'login'
login_manager.init_app(app)

@login_manager.user_loader
def load_user(usuario_id):
  # guarda a chave primária do usuário após login
  return Usuario.query.get(int(usuario_id))

login_manager

### Rotas de acesso

In [ ]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/login')
def login():
    return render_template('login.html')

@app.route('/signup')
def signup():
    return render_template('signup.html')
    
app.url_map

### Retorno dos formulários

In [ ]:
@app.route('/signup', methods=['POST'])
def signup_post():

    email = request.form.get('email')
    usuario = request.form.get('name')
    senha = request.form.get('password')

    # verifica se o usuário já existe no banco
    user = Usuario.query.filter_by(email = email).first() 

    # se existe rediciona a criação para tentar novamente com outro usuário
    if user:
        flash('E-mail de usuário já existe no site')
        return redirect(url_for('signup'))

    # montando dados do usuário para salvar no banco
    new_user = Usuario(
        email = email, 
        usuario = usuario, 
        senha = generate_password_hash(senha, method='sha256'),
        )

    #  inserindo o usuário no banco de dados
    db.session.add(new_user)
    db.session.commit()

    return redirect(url_for('login'))

@app.route('/login', methods=['POST'])
def login_post():

    email = request.form.get('email')
    senha = request.form.get('password')
    remember = True if request.form.get('Lembre-me') else False

    user = Usuario.query.filter_by(email=email).first()

    # se usuário ou hash da senha não são iguais ao do banco
    if not user or not check_password_hash(user.senha, senha):
        flash('Usuário ou senha incorretos')
        return redirect(url_for('login'))

    #cadastra usuário na memória da sessão
    login_user(user, remember=remember)

    # se tudo ok, redireciona a página principal
    return redirect(url_for('profile'))

app.url_map

### Rotas restritas - necessita de login

In [ ]:
@app.route('/profile')
@login_required # proteção de entrada com login
def profile():
    # redireciona a página do usuário com 
    return render_template('profile.html', name = current_user.usuario)

@app.route('/logout')
@login_required # proteção de entrada com login
def logout():
    # descadastra usuário da memória
    logout_user() 
    # rediciona a página inicial da aplicação
    return redirect(url_for('index'))

app.url_map

## Execução do servidor da aplicação

In [ ]:
run_with_ngrok(app)
app.run()